In [ ]:
from components.PdfMemberExtractor import PdfMemberExtractor
import pandas as pd

In [ ]:
extractor = PdfMemberExtractor()
df_pdf = extractor.extract_text(pdf_path="../data/ERC-2025-StG-panel-members.pdf")
df_pdf['ERC-Date'] = df_pdf['ERC-Date'].astype(int)
erc_date = int(df_pdf['ERC-Date'].unique()[0])
display(f"ERC Date: {erc_date}")


# extract review_panel from subdomain
df_pdf['review_panel'] = df_pdf['Subdomain'].str.split('(').str[1].str.replace(')', '').str.strip()

display(df_pdf.head())

Processing: 100%|██████████| 526/526 [00:00<00:00, 6709.07it/s]


'ERC Date: 2025'

,Domain,Subdomain,Forename,Lastname,Fullname,Type,ERC-Date,review_panel
0,PE,Mathematics (PE1),Satoru,Iwata,Satoru Iwata,Chair,2025,PE1
1,PE,Mathematics (PE1),Luigi,Ambrosio,Luigi Ambrosio,Member,2025,PE1
2,PE,Mathematics (PE1),Carolina,Araujo,Carolina Araujo,Member,2025,PE1
3,PE,Mathematics (PE1),Boris,Aronov,Boris Aronov,Member,2025,PE1
4,PE,Mathematics (PE1),Stéphane,Boucheron,Stéphane Boucheron,Member,2025,PE1


In [ ]:
# Read panel-members-excel.xls
excel_path = "../data/panel-members-excel.xls"

df_excel = pd.read_excel(excel_path, engine='openpyxl')

# split column year at "," and copy the row for each year
df_excel = df_excel.assign(year=df_excel['year'].astype(str).str.split(',')).explode('year')

# strip whitespace und entferne leere/ungültige Einträge (nur 4-stellige Jahreszahlen)
df_excel['year'] = df_excel['year'].str.strip()
df_excel = df_excel[df_excel['year'].str.match(r'^\s*\d{4}\s*$', na=False)].copy()

# year in int konvertieren
df_excel['year'] = df_excel['year'].astype(int)



# display only the rows where year == erc_date
df_excel = df_excel[df_excel['year'] == erc_date]

# strip whitespace from Name column
df_excel['Name'] = df_excel['Name'].str.strip()



display(df_excel.head())

,Name,funding_scheme,review_panel,year
61,Deborah Lawlor,SyG,LS,2025
89,Hilleke Hulshoff Pol,SyG,LS,2025
112,Sharon Tooze,SyG,LS,2025
244,Ramesh Pillai,CoG,LS1,2025
274,Rumiana Dimova,StG,LS1,2025


In [ ]:
df_merged = pd.merge(df_pdf, df_excel, left_on='Fullname', right_on='Name', how='inner', indicator=True)

df_merged = df_merged.loc[:, ['Forename', "Lastname", "Fullname", "Type", "year", "review_panel_y", "funding_scheme"]].rename(columns={"review_panel_y": "review_panel"})


display(df_merged.info())
display(df_merged)

print(df_merged["funding_scheme"].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Forename        433 non-null    object
 1   Lastname        433 non-null    object
 2   Fullname        433 non-null    object
 3   Type            433 non-null    object
 4   year            433 non-null    int64 
 5   review_panel    433 non-null    object
 6   funding_scheme  433 non-null    object
dtypes: int64(1), object(6)
memory usage: 23.8+ KB


None

,Forename,Lastname,Fullname,Type,year,review_panel,funding_scheme
0,Satoru,Iwata,Satoru Iwata,Chair,2025,PE1,StG
1,Luigi,Ambrosio,Luigi Ambrosio,Member,2025,PE1,StG
2,Carolina,Araujo,Carolina Araujo,Member,2025,PE1,StG
3,Boris,Aronov,Boris Aronov,Member,2025,PE1,StG
4,Stéphane,Boucheron,Stéphane Boucheron,Member,2025,PE1,StG
...,...,...,...,...,...,...,...
428,Giedrė,Mickūnaitė,Giedrė Mickūnaitė,Member,2025,SH8,StG
429,Toby,Miller,Toby Miller,Member,2025,SH8,StG
430,Nicola,Scaldaferri,Nicola Scaldaferri,Member,2025,SH8,StG
431,Kevin,Smets,Kevin Smets,Member,2025,SH8,StG


funding_scheme
StG    433
Name: count, dtype: int64
